In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

# import _utils # amanda's collected utils

import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils
import datetime as dt
import numpy as np

import gcsfs

from calitp_data.storage import get_fs
fs = get_fs()

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"

import intake

catalog = intake.open_catalog("./metrics_catalog.yml")

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_394/529163112.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  imp

# Safety Metric Demo 
Testing on Watsonville-Santa Cruz multimodal corridor project along SR-1

## Version 1: general project location
This location data comes from the [Rebuilding CA Map](https://rebuildingca.ca.gov/map/)

In [2]:
sample_projects = gpd.read_parquet(f'{GCS_FILE_PATH}nine_sample_projects.parquet')

In [3]:
projloc_sb1 = (sample_projects
               >> filter(_.ct_project_id=="0520000083")
              )

In [5]:
# buffer
projloc_sb1 = projloc_sb1.to_crs(shared_utils.geography_utils.CA_NAD83Albers)
projloc_sb1['b100'] = projloc_sb1.buffer(30)
projloc_sb1 = projloc_sb1.set_geometry('b100')

In [6]:
# load and clip crashes
tims = gpd.read_parquet('gs://calitp-analytics-data/data-analyses/safety_projects/tims_fsi.parquet')

In [7]:
tims_clip = tims.clip(projloc_sb1)
tims_clip = tims_clip.assign(ped_crash = np.where(tims_clip['PEDESTRIAN_ACCIDENT']=="Y",1,0),
                             bike_crash = np.where(tims_clip['BICYCLE_ACCIDENT']=="Y",1,0)
                            )
tims_clip['pedbike_crash'] = tims_clip[["ped_crash", "bike_crash"]].max(axis=1)

In [34]:
projmap = projloc_sb1.explore()
tims_clip.explore(m=projmap, column="COLLISION_SEVERITY")

In [8]:
len(tims_clip)

14

### Demo Crash Reduction Factors:
* Transit signal priority: 14% [cmf clearinghouse reference](https://www.cmfclearinghouse.org/detail.php?facid=11233)
* Mode separation: 41% [ref](https://www.cmfclearinghouse.org/detail.php?facid=2146)
* crosswalks/flashing beacons: 15% [ref](https://www.cmfclearinghouse.org/detail.php?facid=2917)
* Aux lane: 23% [ref](https://www.cmfclearinghouse.org/detail.php?facid=3899)

In [9]:
# combine crash reduction factors - toy example
# reference: https://www.cmfclearinghouse.org/collateral/Combining_Multiple_CMFs_Final.pdf 
# CCRFi = 1 – [(1-CRF1i)*(1-CRF2i)*(1-CRF3i)] 

CRF = 1-((1-0.14)*(1-0.41)*(1-0.15)*(1-0.23))

In [10]:
CRF

0.6679067

In [11]:
CRF*len(tims_clip)

9.3506938

## Version 2: mode separated location
This location data is hand-drawn

In [29]:
# read in shapefiles
projloc_bikeped = catalog.shp_demoproj_bikeped.read()

projloc_bikeped = projloc_bikeped.to_crs(shared_utils.geography_utils.CA_NAD83Albers)

# buffer
projloc_bikeped = projloc_bikeped.to_crs(shared_utils.geography_utils.CA_NAD83Albers)
projloc_bikeped['b100'] = projloc_bikeped.buffer(30)
projloc_bikeped = projloc_bikeped.set_geometry('b100')

# clip TIMS data
tims_pclip = tims.clip(projloc_bikeped)

# map both
projmap = projloc_bikeped.explore(name="bike path")
tims_pclip.explore(m=projmap, color="red")

Repeat for other applicable project widgets

In [26]:
# repeatable function
def moderepeat(catname: str,crf):
    # projloc = catalog.catname.read()
    item = getattr(catalog, catname)
    projloc = item.read()
    projloc = projloc.to_crs(shared_utils.geography_utils.CA_NAD83Albers)
    # projloc.explore()
    projloc['b100'] = projloc.buffer(30)
    projloc = projloc.set_geometry('b100')
    tims_pclip = tims.clip(projloc)
    tims_pclip = tims_pclip.assign(ped_crash = np.where(tims_pclip['PEDESTRIAN_ACCIDENT']=="Y",1,0),
                             bike_crash = np.where(tims_pclip['BICYCLE_ACCIDENT']=="Y",1,0)
                            )
    tims_pclip['pedbike_crash'] = tims_pclip[["ped_crash", "bike_crash"]].max(axis=1)
    print("n FSI =", len(tims_pclip))
    print("CRF =", crf)
    print("Estimated Crashes Reduced =", crf*len(tims_pclip))

In [27]:
moderepeat('shp_demoproj_bikeped', 0.41)

n FSI = 2
CRF = 0.41
Estimated Crashes Reduced = 0.82


In [30]:
moderepeat('shp_demoproj_auxlane', 0.23)

n FSI = 8
CRF = 0.23
Estimated Crashes Reduced = 1.84


In [31]:
moderepeat('shp_demoproj_busshoulder', 0.14)

n FSI = 8
CRF = 0.14
Estimated Crashes Reduced = 1.12


In [32]:
moderepeat('shp_demoproj_multimodal', 0.15)

n FSI = 14
CRF = 0.15
Estimated Crashes Reduced = 2.1
